## Basic Tracking

http://nipy.org/dipy/examples_built/introduction_to_basic_tracking.html#example-introduction-to-basic-tracking

In [1]:
%load_ext autoreload
%autoreload

In [2]:
from dipy.data import read_stanford_labels
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
hardi_img, gtab, labels_img = read_stanford_labels();
data = hardi_img.get_data();
labels = labels_img.get_data();
affine = hardi_img.affine

Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\Seth\.dipy\stanford_hardi 
Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\Seth\.dipy\stanford_hardi 
Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\Seth\.dipy\stanford_hardi 


In [9]:
print(type(gtab))

<class 'dipy.core.gradients.GradientTable'>


In [4]:
white_matter = (labels == 1) | (labels == 2)

In [5]:
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model

csa_model = CsaOdfModel(gtab, sh_order=6)
csa_peaks = peaks_from_model(csa_model, data, default_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=45,
                             mask=white_matter)

In [6]:
from dipy.tracking.local import ThresholdTissueClassifier

classifier = ThresholdTissueClassifier(csa_peaks.gfa, .25)

In [7]:
from dipy.tracking import utils

seed_mask = labels == 2
seeds = utils.seeds_from_mask(seed_mask, density=[2, 2, 2], affine=affine)

In [8]:
from dipy.tracking.local import LocalTracking
from dipy.viz import fvtk
from dipy.viz.colormap import line_colors

# Initialization of LocalTracking. The computation happens in the next step.
streamlines = LocalTracking(csa_peaks, classifier, seeds, affine, step_size=.5)

# Compute streamlines and store as a list.
streamlines = list(streamlines)

# Prepare the display objects.
color = line_colors(streamlines)
print(fvtk.have_vtk)
if fvtk.have_vtk:
    streamlines_actor = fvtk.line(streamlines, line_colors(streamlines))

    # Create the 3D display.
    r = fvtk.ren()
    fvtk.add(r, streamlines_actor)

    # Save still images for this static example. Or for interactivity use
    # fvtk.show
    fvtk.show(r)#.record(r, n_frames=1, out_path='./deterministic.png',
                #size=(800, 800))

True


## Tractography Clustering

http://nipy.org/dipy/examples_built/segment_quickbundles.html#example-segment-quickbundles

In [1]:
import numpy as np
from nibabel import trackvis as tv
from dipy.segment.clustering import QuickBundles
from dipy.io.pickles import save_pickle
from dipy.data import get_data
from dipy.viz import fvtk

In [2]:
fname = get_data('fornix')

In [20]:
streams, hdr = tv.read(fname)
streamlines = [i[0] for i in streams]

In [22]:
print(len(streams))
print(len(streams[0]))
print(len(streams[0][0]))

300
3
79


In [11]:
qb = QuickBundles(threshold=10.)
clusters = qb.cluster(streamlines)

In [19]:
print(len(streamlines))
streamlines[0].shape
#streamlines

300


(79, 3)

In [10]:
print(type(qb))

<class 'dipy.segment.clustering.QuickBundles'>


In [9]:
print("Nb. clusters:", len(clusters))
print("Cluster sizes:", map(len, clusters))
print("Small clusters:", clusters < 10)
print("Streamlines indices of the first cluster:\n", clusters[0].indices)
print("Centroid of the last cluster:\n", clusters[-1].centroid)

Nb. clusters: 4
Cluster sizes: <map object at 0x000000B41873F208>
Small clusters: [False False False  True]
Streamlines indices of the first cluster:
 [0, 7, 8, 10, 11, 12, 13, 14, 15, 18, 26, 30, 33, 35, 41, 65, 66, 85, 100, 101, 105, 115, 116, 119, 122, 123, 124, 125, 126, 128, 129, 135, 139, 142, 143, 144, 148, 151, 159, 167, 175, 180, 181, 185, 200, 208, 210, 224, 237, 246, 249, 251, 256, 267, 270, 280, 284, 293, 296, 297, 299]
Centroid of the last cluster:
 [[  84.83773804  117.92590332   77.32278442]
 [  86.10850525  115.84362793   81.91885376]
 [  86.40357208  112.25676727   85.72930145]
 [  86.48336792  107.60327911   88.13782501]
 [  86.23897552  102.5100708    89.29447174]
 [  85.04563904   97.46020508   88.54240417]
 [  82.60240173   93.14851379   86.84208679]
 [  78.98937225   89.57682037   85.63652039]
 [  74.72344208   86.60827637   84.9391861 ]
 [  70.40846252   85.15874481   82.4484024 ]
 [  66.74534607   86.00262451   78.82582092]
 [  64.02451324   88.43942261   75.069